
<h1><center> <font color='black'>  Network Science (LTAT.02.011) - Homework - 01  </font></center></h1>
<h2><center> <font color='black'> Introduction to Network Analysis</font></center></h3>
<h2><center> <font color='black'> University of Tartu - Spring 2023</font></center></h3>


================================================================================================================

# Homework instructions

- This homework should be done individually or in group of 2 (max).

- Please provide the names and student IDs of the team-members in the field "Student info" below. If you are not working in a team please insert only your name and student ID. 

- Only one of the teammates should submit the homework. We will grade the homework and the marks and feedback is applied for both the team members. So please communicate with your team member about marks and feedback if you are submitting the homework.

- The accepted submission format is .ipynb file. Please upload the file on moodle.

- The submission will automatically close on **<font color='red'>22 March 2023 at 23:59 pm</font>**, so please make sure you have enough time to submit the homework. 

- You do not necessarily need to work on Colab. Especially as the size and the complexity of datasets will increase through the course, you can install jupyter notebooks locally and work from there.

- If you do not understand what a question is asking for, please ask in Moodle. 


**<h2><font color='red'>Student info:</font></h2>**


<font color='red'>Full name: </font> Samir Musali

<font color='red'>Student ID: </font>C29810

# Dataset details:
**The network data consists of trading countries. There are two files. The first file, *nodes.csv* file has information about the countries where each row consists of node id and the corresponding country name. The second file *edges.csv* contains information about trading countries. Each row represents two countries which trade which each other.**



# Libraries


In [1]:
# Installing additional dependencies
!pip install cairocffi
!pip install python-igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cairocffi: filename=cairocffi-1.5.0-py3-none-any.whl size=90538 sha256=078b7d168e13820df1b3fe87cbe508b4cfa7b65c7dd97f668f819aadc011be72
  Stored in directory: /root/.cache/pip/wheels/34/c0/21/58ab88c0e5a36a32f10f5257afbd6e5fae805171da13691863
Successfully built cairocffi
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.2 MB/s eta 0:00:00
  Created wheel for python-igraph: filename=python_igraph-0.10.4-py3-none-any.whl size=9073 sha256=114d8a964a4973a268a48f89e6792dc77

In [2]:
# Importing libraries
import igraph as ig
import networkx as nx
from networkx.algorithms import community
import pandas as pd
import matplotlib.pyplot as plt

# Reading the datasets

In [3]:
nodes = pd.read_csv('nodes.csv')
edges = pd.read_csv('edges.csv')

In [4]:
print('Nodes dataset dimension:', nodes.shape, '\nEdges dataset dimension:', edges.shape)

Nodes dataset dimension: (214, 2) 
Edges dataset dimension: (318346, 2)


In [5]:
nodes.head()

,id,Name
0,1,Afghanistan
1,2,Australia
2,3,Austria
3,4,Belgium
4,5,Brazil


In [6]:
edges.head()

,from,to
0,2,1
1,2,2
2,2,3
3,2,4
4,2,6


# 1. Network creation (1.5 points)

**1.1. Create undirected and unweighted network using nodes and edges datasets (0.25 point)**

<font color='red'>How you should answer the following questions: In the `#TODO` cell you have to write the code representing your solution.</font>

In [7]:
"""
Hint: Use igraph or networkx to create a correct graph type. 
"""
G_unweighted_undirected = nx.Graph()
for _, row in nodes.iterrows():
    G_unweighted_undirected.add_node(row['id'], attr_dict=row.to_dict())

for _, row in edges.iterrows():
    G_unweighted_undirected.add_edge(row['from'], row['to'])

**1.2. Create undirected and weighted network using nodes and edges datasets (0.25 point)**

In [9]:
'''
Hint: Create "weight" column in the "edges" dataframe by grouping on "from" and "to" columns.
'''
edge_weights = edges.groupby(['from', 'to']).size()
edges['weight'] = edges.apply(lambda row: edge_weights[row['from'], row['to']], axis=1)
G_weighted_undirected = nx.Graph()
for _, row in nodes.iterrows():
    G_weighted_undirected.add_node(row['id'], attr_dict=row.to_dict())
for _, row in edges.iterrows():
    G_weighted_undirected.add_edge(row['from'], row['to'], weight=row['weight'])

**1.3. Create directed and unweighted network using nodes and edges datasets (0.25 point)**

In [10]:
G_unweighted_directed = nx.DiGraph()
for _, row in nodes.iterrows():
    G_unweighted_directed.add_node(row['id'], attr_dict=row.to_dict())
for _, row in edges.iterrows():
    G_unweighted_directed.add_edge(row['from'], row['to'])

**1.4. Create directed and weighted network using nodes and edges datasets (0.25 point)**

In [11]:
G_weighted_directed = nx.DiGraph()
for _, row in nodes.iterrows():
    G_weighted_directed.add_node(row['id'], attr_dict=row.to_dict())
for _, row in edges.iterrows():
    G_weighted_directed.add_edge(row['from'], row['to'], weight=row['weight'])

**1.5. Check whether undirected and unweighted network is connected or disconected (0.5 point).**

<font color='red'>How you should answer the following questions. First, in the `#TODO` cell you have to write the code representing your solution. And in the `Your answer` cell write the answer in your own words. It does not have to be a full sentence.</font>

In [12]:
is_connected = nx.is_connected(G_unweighted_undirected)

if is_connected:
    print("The graph is connected")
else:
    print("The graph is disconnected")

The graph is connected


**<font color='red'>Your answer:</font>** Apparently, it's connected!

# 2. Network properties (3 points)

## 2.1. Edge density (0.5 points).

**2.1.1. Calculate edge density for undirected and unweighted network (0.125 point);**

In [13]:
n_nodes = G_unweighted_undirected.number_of_nodes()
n_edges = G_unweighted_undirected.number_of_edges()
max_edges = (n_nodes * (n_nodes - 1)) / 2
edge_density = n_edges / max_edges

print("Edge density:", edge_density)

Edge density: 0.41424246413057786


**<font color='red'>Your answer:</font>** ~0.41

**2.1.2. Calculate edge density for directed and unweighted network (0.125 point)**

In [14]:
n_nodes = G_unweighted_directed.number_of_nodes()
n_edges = G_unweighted_directed.number_of_edges()
max_edges = n_nodes * (n_nodes - 1)
edge_density = n_edges / max_edges

print("Edge density:", edge_density)

Edge density: 0.30134702294765475


**<font color='red'>Your answer:</font>** ~0.3

**2.1.3 Compare the results you received. Which network has a higher edge density? Why? (0.25 point)**

**<font color='red'>Your answer:</font>** As you can see, the edge density of the directed and unweighted network is lower since the maximum possible number of edges is more in directed network than in undirected one.

## 2.2. Clustering coefficient (1 point)

**2.2.1. Calculate average clustering coefficient for undirected and unweighted network (0.125 point);**

In [15]:
average_clustering_coefficient = nx.average_clustering(G_unweighted_undirected)
print("Average clustering coefficient:", average_clustering_coefficient)

Average clustering coefficient: 0.8124991062175771


**<font color='red'>Your answer:</font>** ~0.81

**2.2.2. Calculate average clustering coefficient for directed and unweighted network (0.125 point);**

In [16]:
average_clustering_coefficient = nx.average_clustering(G_unweighted_directed, count_zeros=True)
print("Average clustering coefficient:", average_clustering_coefficient)

Average clustering coefficient: 0.7480679346326201


**<font color='red'>Your answer:</font>** ~0.75

**2.2.3 Compare the results you received for unweighted networks. Which network has a higher average clustering coefficient? Why? (0.25 point);**

**<font color='red'>Your answer:</font>** The average clustering coefficient of `G_unweighted_directed` is lower than that of `G_unweighted_undirected` because the directed edges in `G_unweighted_directed` do not allow for the formation of the same kind of clusters as in the undirected graph.

**2.2.4. Calculate average clustering coefficient for undirected and weighted network (0.125 point);**

In [17]:
avg_cluster_coeff = nx.average_clustering(G_weighted_undirected)
print("Average clustering coefficient for G_weighted_undirected: ", avg_cluster_coeff)

Average clustering coefficient for G_weighted_undirected:  0.8124991062175771


**<font color='red'>Your answer:</font>** ~0.81

**2.2.5. Calculate average clustering coefficient for directed and weighted network (0.125 point);**

In [18]:
avg_cluster_coeff = nx.average_clustering(G_weighted_directed)
print("Average clustering coefficient for G_weighted_directed: ", avg_cluster_coeff)

Average clustering coefficient for G_weighted_directed:  0.7480679346326201


**<font color='red'>Your answer:</font>** ~0.75

**2.2.6 Compare the results you received for weighted networks. Which network has a higher average clustering coefficient? Why? (0.25 point);**

**<font color='red'>Your answer:</font>** In `G_weighted_directed`, since the edges have weights, the definition of clustering coefficient is modified to take into account the weights of the edges. However, since the graph is directed, this means that the incoming and outgoing edges are treated separately, leading to a lower clustering coefficient compared to an undirected graph with the same set of edges.

In contrast, in `G_weighted_undirected`, the graph is undirected, and the clustering coefficient is calculated based on the connections between the neighbors of each node, without taking into account the direction of the edges. Therefore, the clustering coefficient for `G_weighted_undirected` will generally be higher than for a directed network with the same set of edges.

## 2.3. Diameter (0.5 point)

**2.3.1. Calculate diameter for undirected and unweighted network (0.125 point); and interpret the result (0.125 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**2.3.2. Calculate diameter for directed and unweighted network (0.125 point); and interpret the result (0.125 point)**

In [19]:
'''
Hint: This will generate an error. Please find and explain the reason in your own words and suggest a possible solution.
'''
diameter = nx.diameter(G_unweighted_directed)
print("Diameter of G_unweighted_directed:", diameter)

NetworkXError: ignored

[For further help](https://stackoverflow.com/questions/33114746/why-does-networkx-say-my-directed-graph-is-disconnected-when-finding-diameter)

**<font color='red'>Your answer:</font>** The error message suggests that the `nx.diameter` function could not calculate the diameter of the directed and unweighted network `G_unweighted_directed` because it is not strongly connected. The diameter is defined as the maximum shortest path between any two nodes in a graph. If a graph is not strongly connected, meaning there is at least one node that cannot be reached from every other node, then it is impossible to compute the shortest path between some pairs of nodes, resulting in an infinite path length and thus, the `NetworkXError`.

## 2.4. Average path length (1 point)

**2.4.1. Calculate average path lenth for undirected and unweighted network (0.125 point);**

In [22]:
avg_path_length = nx.average_shortest_path_length(G_unweighted_undirected)
print("Average path length of G_unweighted_undirected:", avg_path_length)

Average path length of G_unweighted_undirected: 1.601246105919003


**<font color='red'>Your answer:</font>** ~1.6

**2.4.2. Calculate average path lenth for directed and unweighted network (0.125 point);**

In [20]:
components = nx.strongly_connected_components(G_unweighted_directed)
avg_path_lengths = []
for c in components:
    subgraph = G_unweighted_directed.subgraph(c)
    if len(subgraph) > 1:
        avg_path_lengths.append(nx.average_shortest_path_length(subgraph))

avg_path_length = sum(avg_path_lengths) / len(avg_path_lengths)
print("Average path length of G_unweighted_directed:", avg_path_length)

Average path length of G_unweighted_directed: 1.3847117794486214


**<font color='red'>Your answer:</font>** ~1.39

**2.4.3 Compare the results you received for unweighted networks. Which network has a higher average path length? Why? (0.25 point);**

**<font color='red'>Your answer:</font>**

**2.4.4. Calculate average path lenth for undirected and weighted network (0.125 point);**

In [23]:
avg_path_length = nx.average_shortest_path_length(G_weighted_undirected)
print("Average path length of G_weighted_undirected:", avg_path_length)

Average path length of G_weighted_undirected: 1.601246105919003


**<font color='red'>Your answer:</font>** ~1.6

**2.4.5. Calculate average path lenth for directed and weighted network (0.125 point);**

In [24]:
components = nx.strongly_connected_components(G_weighted_directed)
avg_path_lengths = []
for c in components:
    subgraph = G_weighted_directed.subgraph(c)
    if len(subgraph) > 1:
        avg_path_lengths.append(nx.average_shortest_path_length(subgraph))

avg_path_length = sum(avg_path_lengths) / len(avg_path_lengths)
print("Average path length of G_weighted_directed:", avg_path_length)

Average path length of G_weighted_directed: 1.3847117794486214


**<font color='red'>Your answer:</font>** ~1.39

**2.4.6 Compare the results you received for weighted networks with unweighted networks. Which network type (unweighted vs weighted)  has a higher average path length? Why? (0.25 point);**

**<font color='red'>Your answer:</font>**

# 3. Network centrality measures (2 points)

## 3.1. Degree centrality (0.5 points).

**3.1.1. Find the top 4 countries based on degree centrality for undirected and unweighted network (0.25 point).**

In [ ]:
'''
Hint: In case our network is created using edgelist, we need to map it with the actual country name provided in the "nodes" dataframe.
'''
#TODO

**<font color='red'>Your answer:</font>**

**3.1.2. Find the top 4 countries based on degree centrality for directed and unweighted network (0.25 point).**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

## 3.2. Closeness centrality (0.5 points).

**3.2.1. Find the top 4 countries based on closeness centrality for undirected and weighted network (0.25 point)**

In [ ]:
'''
Hint: Create 'distance' using 'weight' feature. Refer lab 04 for detail.
'''
#TODO

**<font color='red'>Your answer:</font>**

**3.2.2. Find the top 4 countries based on closeness centrality for directed and weighted network (0.25 point)**

In [ ]:
'''
Hint: Create 'distance' using 'weight' feature. Refer lab 04 for detail.
'''
#TODO

**<font color='red'>Your answer:</font>**

## 3.3. Betweenness centrality (0.5 points).

**3.3.1. Find the top 4 countries based on Betweenness centrality for undirected and unweighted network (0.25 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**3.3.2. Plotted the undirected and unweighted network by highlighting the most important nodes (according to betweenness centrality) 
using different size and/or color (0.25 point)**

In [ ]:
#TODO

## 3.4. Eigenvector centrality (0.5 points).

**3.4.1. Find the top 4 countries based on Eigenvector centrality for undirected and unweighted network (0.25 point).**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**3.4.2. Find the top 4 countries based on Eigenvector centrality for directed and weighted network (0.225 point).**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

# 4. Community detection (2 points)

## 4.1. Girvan–Newman (Edge betweenness) (0.5 points)

**4.1.1. Report the optimal number of communities you find using Girvan–Newman algorithm for undirected and unweighted network (0.25 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>** 

**4.1.2. Plotted the undirected and unweighted network by highlighting the nodes in various communities (according to the optimal number of communities based on Girvan–Newman algorithm) using different color (0.25 point).**

In [ ]:
#TODO

## 4.2. Louvian (0.5 points)

**4.2.1. Report the number of communities you find using Louvian algorithm for undirected and unweighted network (0.25 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**4.2.2. Plotted the undirected and unweighted network by highlighting the nodes in various communities (according to Louvian algorithm) using different color (0.25 point).**

In [ ]:
#TODO

## 4.3. Walk Trap (0.5 points)

**4.3.1. Report the number of communities you find using Walk Trap algorithm (with step size = 4) for undirected and unweighted network (0.25 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**4.3.2. Plotted the undirected and unweighted network by highlighting the nodes in various communities (according to Walk Trap algorithm with step size = 4) using different color (0.25 point).**

In [ ]:
#TODO

## 4.4. Label propagation (0.5 points)

**4.4.1. Report the number of communities you find using Label propagation algorithm for undirected and unweighted network (0.25 point)**

In [ ]:
#TODO

**<font color='red'>Your answer:</font>**

**4.4.2. Plotted the undirected and unweighted network by highlighting the nodes in various communities (according to Label propagation algorithm) using different color (0.25 point).**

In [ ]:
#TODO

# 5. Network Visualization Using Gephi (1.5 points)

**5.1. Create graph using gephi. Adjust the node size based on degree centrality (for undirected network) and color them based on modularity value. Please import your network as png and attach to the notebook (0.5 point)**

<font color='red'>Please note that you don't need to upload the png file. You can insert it and save the notebook and it will be available to us.</font>

In [ ]:
from IPython.display import Image
Image("ImageName.png")

**5.2. Create graph using gephi. Adjust the node size based on PageRank centrality (for undirected network) and color them based on modularity value. Please import your network as png and attach to the notebook (0.5 point)**

In [ ]:
#Image("ImageName.png")

**5.3. Create graph using gephi. Adjust the node size based on eigenvector centality (for undirected network) and color them based on modularity value. Please import your network as png and attach to the notebook (0.5 point)**

In [ ]:
#Image("ImageName.png")

## How long did it take you to solve the homework?

* Please answer as precisely as you can. It does not affect your points or grade in any way. It is okay, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks. Please change X into your estimate.

<font color='red'> **Answer:**</font> X hours 

## What is the level of difficulty for this homework?
Please put a number between $0:10$ ($0:$ easy, $10:$ difficult)

<font color='red'> **Answer:**</font>